<a href="https://colab.research.google.com/github/shubhaan06/Clustering/blob/main/Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
# Clustering Assignment - Iris Dataset (with DBSCAN instead of Mean Shift)
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

In [23]:
# Load dataset
data = load_iris()
X = data.data
df = pd.DataFrame(X, columns=data.feature_names)

In [24]:
# Define Preprocessing Functions
def normalize(X):
    return MinMaxScaler().fit_transform(X)

def standardize(X):
    return StandardScaler().fit_transform(X)

def apply_pca(X, n_components=2):
    return PCA(n_components=n_components).fit_transform(X)

def log_transform(X):
    return np.log1p(X)

In [25]:
# Define evaluation function
def evaluate_clustering(X, labels):
    if len(set(labels)) <= 1 or len(set(labels)) == len(labels):
        return {'Silhouette': 0, 'Calinski-Harabasz': 0, 'Davies-Bouldin': 999}
    return {
        'Silhouette': round(silhouette_score(X, labels), 2),
        'Calinski-Harabasz': round(calinski_harabasz_score(X, labels), 2),
        'Davies-Bouldin': round(davies_bouldin_score(X, labels), 2)
    }

In [26]:
# Preprocessing scenarios
scenarios = {
    "Raw": lambda X: X,
    "Normalized": normalize,
    "Log Transformed": log_transform,
    "PCA": lambda X: apply_pca(X),
    "T+N": lambda X: normalize(log_transform(X)),
    "T+N+PCA": lambda X: apply_pca(normalize(log_transform(X)))
}

In [27]:
clustering_algorithms = {
    "KMeans": lambda k: KMeans(n_clusters=k, random_state=42),
    "Hierarchical": lambda k: AgglomerativeClustering(n_clusters=k),
    "GMM": lambda k: GaussianMixture(n_components=k, random_state=42)
}

In [28]:
# Result collector
results = []

In [29]:
for method_name, preprocess_func in scenarios.items():
    X_proc = preprocess_func(df.values)
    for algo_name, algo_func in clustering_algorithms.items():
        cluster_range = [3, 4, 5]  # Same range for all algorithms now
        for k in cluster_range:
            model = algo_func(k)
            if algo_name == "GMM":
                labels = model.fit_predict(X_proc)  # GMM uses fit_predict
            else:
                labels = model.fit_predict(X_proc)
            scores = evaluate_clustering(X_proc, labels)
            scores.update({
                "Preprocessing": method_name,
                "Algorithm": algo_name,
                "Clusters": k
            })
            results.append(scores)

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Pivot Table View
pivot_table = pd.pivot_table(results_df,
                              values=['Silhouette', 'Calinski-Harabasz', 'Davies-Bouldin'],
                              index=['Algorithm', 'Preprocessing', 'Clusters'])
print("\n=== Evaluation Results ===")
display(pivot_table)


=== Evaluation Results ===


Calinski-Harabasz  Davies-Bouldin  \
Algorithm    Preprocessing   Clusters                                      
GMM          Log Transformed 3                    500.82            0.99   
                             4                    655.57            1.11   
                             5                    634.48            1.26   
             Normalized      3                    308.79            0.87   
                             4                    243.69            1.18   
                             5                    213.87            0.99   
             PCA             3                    563.92            0.67   
                             4                    485.75            0.80   
                             5                    544.28            0.75   
             Raw             3                    481.78            0.75   
                             4                    329.58            1.81   
                             5                    441.63            0.92   
             T+N             3                    359.59            0.98   
                             4                    267.68            1.21   
                             5                    279.47            1.29   
             T+N+PCA         3                    327.86            0.62   
                             4                    343.03            0.95   
                             5                    339.68            1.10   
Hierarchical Log Transformed 3                    974.18            0.63   
                             4                    786.66            0.72   
                             5                    671.50            0.84   
             Normalized      3                    349.25            0.75   
                             4                    301.10            0.85   
                             5                    272.02            0.91   
             PCA             3                    688.62            0.56   
                             4                    673.95            0.65   
                             5                    665.88            0.65   
             Raw             3                    558.06            0.66   
                             4                    515.08            0.80   
                             5                    488.48            0.82   
             T+N             3                    406.37            0.77   
                             4                    347.70            0.88   
                             5                    323.14            1.00   
             T+N+PCA         3                    504.33            0.59   
                             4                    479.35            0.62   
                             5                    476.46            0.72   
KMeans       Log Transformed 3                    502.49            0.94   
                             4                    721.75            1.03   
                             5                    715.17            1.07   
             Normalized      3                    351.30            0.79   
                             4                    313.89            0.91   
                             5                    263.17            0.99   
             PCA             3                    693.71            0.56   
                             4                    719.12            0.62   
                             5                    642.06            0.75   
             Raw             3                    561.59            0.67   
                             4                    529.53            0.75   
                             5                    495.24            0.82   
             T+N             3                    439.85            0.78   
                             4                    353.51            0.95   
                             5                    342.69            1.01   
           

In [30]:
# Normalize and Find Best Configuration
normalized = results_df.copy()
normalized['Silhouette_N'] = (normalized['Silhouette'] - normalized['Silhouette'].min()) / (normalized['Silhouette'].max() - normalized['Silhouette'].min())
normalized['CH_N'] = (normalized['Calinski-Harabasz'] - normalized['Calinski-Harabasz'].min()) / (normalized['Calinski-Harabasz'].max() - normalized['Calinski-Harabasz'].min())
normalized['DB_N'] = 1 - (normalized['Davies-Bouldin'] - normalized['Davies-Bouldin'].min()) / (normalized['Davies-Bouldin'].max() - normalized['Davies-Bouldin'].min())
normalized['Overall_Score'] = (normalized['Silhouette_N'] + normalized['CH_N'] + normalized['DB_N']) / 3

best_row = normalized.sort_values(by='Overall_Score', ascending=False).iloc[0]
print("\n=== 🔍 Best Clustering Configuration ===")
print(f"Algorithm     : {best_row['Algorithm']}")
print(f"Preprocessing : {best_row['Preprocessing']}")
print(f"Clusters      : {best_row['Clusters']}")
print(f"Silhouette    : {best_row['Silhouette']}")
print(f"Calinski-Harabasz : {best_row['Calinski-Harabasz']}")
print(f"Davies-Bouldin    : {best_row['Davies-Bouldin']}")
print(f"Overall Score     : {round(best_row['Overall_Score'], 3)}")


=== 🔍 Best Clustering Configuration ===
Algorithm     : Hierarchical
Preprocessing : Log Transformed
Clusters      : 3
Silhouette    : 0.57
Calinski-Harabasz : 974.18
Davies-Bouldin    : 0.63
Overall Score     : 0.954
